In [26]:
# this script takes custom inputs of stock market parameters
# simulate the stock market using Monte Carlo simulation
# compare overall profolio after a set number of years when
# buying using daily cost average method or buying at the dip (but at -10% of all thhe high)

In [27]:
import datetime
import math
import os
import shutil

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_style("whitegrid")

In [28]:
# change to a local folder to save data
parent_dir = os.getcwd()  # r"C:\Users\milkopals\Documents\Python Scripts\MonteCarlo"

In [29]:
## set up run mode. details see Cell 8
# TEST: less simulation cycles, less day in a year, save to TEST folder.
# CUSTOM: use custom parameters.
# PROD: use production default parameters.

run_mode = "TEST"  # "CUSTOM" , "PROD"

if run_mode == "CUSTOM":  # change here for custom parameters

    # number of times for monte carlo simulation for stock price function
    NUM_SIMULATIONS = 10

    # Stock market parmaeters: day & year definition
    NUM_YEARS = 20
    TRADING_DAYS_IN_YEAR = 250

    # average growth per year: in actual number i.e. 5% = 5 in range
    growth_per_year = [(x + 1) / 100 for x in list(range(5, 10, 1))]

    # frequency to buy daily cost average
    DCA_INTERVAL_DAY = 25

    # figure related parameters
    PLOT_FIGURE = 0
    CLOSE_FIGURE = 0

elif run_mode == "TEST":
    NUM_SIMULATIONS = 1
    NUM_YEARS = 10
    TRADING_DAYS_IN_YEAR = 20
    growth_per_year = [(x + 1) / 100 for x in list(range(14, 15, 1))]
    DCA_INTERVAL_DAY = 2

    PLOT_FIGURE = 0
    CLOSE_FIGURE = 0

elif run_mode == "PROD":
    NUM_SIMULATIONS = 100
    NUM_YEARS = 20
    TRADING_DAYS_IN_YEAR = 250
    growth_per_year = [(x + 1) / 100 for x in list(range(0, 15, 1))]
    DCA_INTERVAL_DAY = 25

    PLOT_FIGURE = 1
    CLOSE_FIGURE = 1

In [30]:
## set up other stock markets and profolio related parameters
# based on historic yearly data to disturb the stock market upward trend
STOCK_STD_DEV = 0.149

START_MARKET_PRICE = 100
START_PROFOLIO_AMOUNT = 1000
DCA_AMOUNT = 100

In [31]:
## market crashes
## statistics done separaetly from analyzing revious stock market behaviours
## not included in this code
CRASH_ENABLE = 1

# number and their probabilities between two stock market crashes
# round to year; >10% stock price drop is defined as a crash
crash_years = [1, 2, 3, 4, 7, 8]
crash_year_prob = [0.4815, 0.2593, 0.0741, 0.1111, 0.037, 0.037]

# % drop and their probabilities between two stock market crashes
# use 10% interval, -0.1 = stock price drop 10% from previous all time high
crash_values = [-0.1, -0.2, -0.3, -0.5, -0.6]
crash_value_prob = [0.4074, 0.3704, 0.1111, 0.0741, 0.037]

# for each crash_vaules, the time and probability for the market to come back to the previous all time high.
# example: when the market drops -10%, it takes less time for the market to come back to previous high (0.5-1 year)
# compared with when the market drops -60% (4 year)
improve_time = {
    -0.1: [0.5, 1],
    -0.2: [0.5, 1.5, 2],
    -0.3: [0.5, 2],
    -0.5: [4.5, 6],
    -0.6: [4],
}

improve_prob = {
    -0.1: [0.9, 0.1],
    -0.2: [0.8, 0.1, 0.1],
    -0.3: [0.7, 0.3],
    -0.5: [0.5, 0.5],
    -0.6: [1],
}

In [32]:
# calculate final share and cost basis from added share and added amount
def final_share_cost_basis_cal(df_dataframe):
    for row_k in range(df_dataframe.shape[0]):
        if df_dataframe.iloc[row_k]["add_amount"] > 0:
            df_dataframe.at[row_k, "final_share"] = (
                df_dataframe.iloc[row_k - 1]["final_share"]
                + df_dataframe.iloc[row_k]["add_share"]
            )
            df_dataframe.at[row_k, "cost_basis"] = (
                df_dataframe.iloc[row_k - 1]["cost_basis"]
                + df_dataframe.at[row_k, "add_amount"]
            )
        else:
            df_dataframe.at[row_k, "final_share"] = df_dataframe.iloc[row_k - 1][
                "final_share"
            ]
            df_dataframe.at[row_k, "cost_basis"] = df_dataframe.iloc[row_k - 1][
                "cost_basis"
            ]
    return df_dataframe

In [33]:
# calculate overall gain through the period and save them in a list
def save_overall_gain(df_dataframe, overall_gain_sum_list):
    overall_gain = (
        df_dataframe.iloc[-1]["final_asset"] - df_dataframe.iloc[-1]["cost_basis"]
    ) / df_dataframe.iloc[-1]["cost_basis"]

    overall_gain_sum_list.append(overall_gain)

    return overall_gain_sum_list

In [34]:
## buy the dip specific functions

# fill in 'add_amount' column on the crash data by sum up available cash if using dca method till the crash day
def dip_add_amount(df_dataframe, crash_day_target2_list, total_trading_days_int):
    for ck, currentcrash in enumerate(crash_day_target2_list):
        if ck == 0:
            if currentcrash == 0:
                amount_till_now = 0
            else:
                amount_till_now = sum(df_dataframe["add_amount_temp"][:currentcrash])
        else:
            prevcrash = crash_day_target2_list[ck - 1]
            amount_till_now = sum(
                df_dataframe["add_amount_temp"][prevcrash:currentcrash]
            )
        df_dataframe.at[currentcrash, "add_amount"] = amount_till_now

    # configure last day 'add_amount' by substracting all newly calculated dip add_amount (add_amount) from dca add mount (add_amount_temp)
    if df_dataframe.iloc[total_trading_days_int - 1]["add_amount"] == 0:
        remaining = sum(df_dataframe["add_amount_temp"]) - sum(
            df_dataframe["add_amount"]
        )
        df_dataframe.at[total_trading_days - 1, "add_amount"] = remaining

    return df_dataframe

In [35]:
# calculate num of shares bought when crash based on 'add_amount'
# the purchase price is be a function of previous ath (simulatie limit buy)
def dip_add_share_calc(df_dataframe, buy_price_relative_ath):
    for row_k in range(df_dataframe.shape[0]):
        if (
            row_k == df_dataframe.shape[0] - 1
        ):  # if any amount left on last day, all in to buy
            df_dataframe.at[row_k, "add_share"] = df_dataframe.iloc[row_k][
                "add_amount"
            ] / (df_dataframe.iloc[row_k]["price"])
        else:  # on other days, only buy with available cash at previous day price * buy_price_relative_ath
            df_dataframe.at[row_k, "add_share"] = df_dataframe.iloc[row_k][
                "add_amount"
            ] / (df_dataframe.iloc[row_k - 1]["price"] * buy_price_relative_ath)

    # cash remaining at hand= sum (dac amounts till today) - sum (dip amounts till today)
    for row_k in range(df_dataframe.shape[0]):
        if row_k == 0:
            df_dataframe.at[row_k, "cash"] = df_dataframe["add_amount_temp"][0]
        else:
            df_dataframe.at[row_k, "cash"] = sum(
                df_dataframe["add_amount_temp"][: row_k + 1]
            ) - sum(df_dataframe["add_amount"][: row_k + 1])

    df_dataframe["final_asset"] = (
        df_dataframe["final_share"] * df_dataframe["price"] + df_dataframe["cash"]
    )

    return df_dataframe

In [36]:
# calculate final table of buy the dip method
def dip_final(
    df_dataframe,
    crash_day_target2_list,
    total_trading_days_int,
    buy_price_relative_ath,
    overall_gain_sum_list,
):

    dip_add_amount(df_dataframe, crash_day_target2_list, total_trading_days_int)

    dip_add_share_calc(df_dataframe, buy_price_relative_ath)

    final_share_cost_basis_cal(df_dataframe)

    # clean up the data table
    df_dataframe["mode"] = "dip"

    save_overall_gain(df_dataframe, overall_gain_sum_list)

    return df_dataframe, overall_gain_sum_list

In [37]:
## buy the dip specific functions

# fill in 'add_amount' column on the intended dca day
def dca_add_amount(df_dataframe, DCA_INTERVAL_DAY_CONS, DCA_AMOUNT_CONS):
    for row_k in range(df_dataframe.shape[0]):
        if (row_k + 1) % DCA_INTERVAL_DAY_CONS == 0:
            df_dataframe.at[row_k, "add_amount"] = DCA_AMOUNT_CONS
            df_dataframe.at[row_k, "add_share"] = (
                df_dataframe.iloc[row_k]["add_amount"]
                / df_dataframe.iloc[row_k]["price"]
            )
    return df_dataframe

In [38]:
## daily cost average final table: calculate shares bought, final share, cost basis and final assest parameters
def dca_final(
    df_dataframe, DCA_INTERVAL_DAY_CONS, DCA_AMOUNT_CONS, overall_gain_sum_list
):

    dca_add_amount(df_dataframe, DCA_INTERVAL_DAY_CONS, DCA_AMOUNT_CONS)

    final_share_cost_basis_cal(df_dataframe)

    df_dataframe["mode"] = "dca"
    df_dataframe["final_asset"] = df_dataframe["final_share"] * df_dataframe["price"]

    save_overall_gain(df_dataframe, overall_gain_sum_list)

    return df_dataframe, overall_gain_sum_list

In [39]:
## calculate apr per year during the simulation period
# !!!!! CHECK THE CALCULATION !!!!!
def aprcalc(df_dataframe, TRADING_DAYS_IN_YEAR_CONS, current_simulation_int):
    apr_sum = []
    apr = []
    for row_k in range(df_dataframe.shape[0]):
        if (row_k + 1) % TRADING_DAYS_IN_YEAR_CONS == 0:
            aprrate = (
                df_dataframe.iloc[row_k]["final_asset"]
                - df_dataframe.iloc[row_k]["cost_basis"]
            ) / df_dataframe.iloc[row_k]["cost_basis"]
            apr.append(aprrate)

    dapr = pd.DataFrame(
        index=range(NUM_YEARS),
        data={
            "year": range(NUM_YEARS),
            "apr": apr,
            "num_simulation": current_simulation_int,
        },
    )

    daprdesc = dapr.describe()["apr"]
    dapr2 = daprdesc.to_frame().iloc[1:]
    dapr2["num_simulation"] = current_simulation_int
    apr_sum.append(dapr2)
    dfapr_dataframe = pd.concat(apr_sum)

    return dfapr_dataframe

In [40]:
def plot_figs(dftotal_dataframe, x_axis_string, yaxis_string, path_dir):
    plt.figure()
    sns.lineplot(data=dftotal_dataframe, x=x_axis_string, y=yaxis_string, hue="mode")
    write_log(logfile, "finish " + yaxis_string + " plot")
    plotnames = path_dir + "\\" + yaxis_string + ".png"
    plt.savefig(plotnames)

In [41]:
# figures set-up back up
if PLOT_FIGURE == 0:
    CLOSE_FIGURE = 1

In [42]:
## set up folders to save based on run_mode
if run_mode == "TEST":  # delete the test folder and make an new one
    parent_dir = parent_dir + "\\TEST"
    if os.path.isdir(parent_dir):
        shutil.rmtree(parent_dir)
    os.makedirs(parent_dir)

else:  # make a new folder based on current time
    now = datetime.datetime.now()
    current_time = now.strftime("%Y/%m/%d %H:%M:%S")
    current_time = current_time.replace("/", "").replace(":", "").replace(" ", "_")
    parent_dir = parent_dir + "\\data"
    parent_dir = os.path.join(parent_dir, current_time)
    if not os.path.isdir(parent_dir):
        os.makedirs(parent_dir)

In [43]:
# writing a log to keep track
logfile = "log.log"

def write_log(logfile_file, text_str):
    now_time = datetime.datetime.now()
    current_time_str = now_time.strftime("%m/%d/%Y %H:%M:%S")
    text_str = current_time_str + "  " + text_str
    with open(logfile_file, "a") as file:
        file.write(text_str)
        file.write("\n")

In [44]:
now = datetime.datetime.now()
current_time = now.strftime("%m/%d/%Y %H:%M:%S")
text = (
    current_time
    + "  start: TRADING_DAYS_IN_YEAR = "
    + str(TRADING_DAYS_IN_YEAR)
    + "; NUM_YEARS = "
    + str(NUM_YEARS)
    + "; DCA_INTERVAL_DAY = "
    + str(DCA_INTERVAL_DAY)
)

with open(logfile, "w") as f:
    f.write(text)
    f.write("\n")

growth_per_day = [x / TRADING_DAYS_IN_YEAR for x in growth_per_year]
total_trading_days = NUM_YEARS * TRADING_DAYS_IN_YEAR

In [45]:
## stock market price = price * stdev (disturbance around the price) * crash_function * growth_function (how to recover from crash)

In [46]:
def improve_time_speed_based_on_crash_value(
    improve_time_dict,
    improve_prob_dict,
    TRADING_DAYS_IN_YEAR_CONS,
    crash_years_list,
    crash_year_prob_list,
    NUM_YEARS_CONS,
    crash_value_prob_list,
    crash_values_list
):
    crash_target_list = list(
        np.random.choice(
            crash_values_list, math.ceil(NUM_YEARS_CONS / 0.5), p=crash_value_prob_list
        )
    )

    finish_days_list = []
    improve_rates_list = []
    crash_year_target_list = []
    improve_back_list = []

    # based on crash target find the improve time and associated probability
    for ci, target in enumerate(crash_target_list):
        ibt = improve_time_dict.get(target)
        ibp = improve_prob_dict.get(target)

        # based on the probability of the crash target , generate days needed to come back to previous high
        if len(ibp) > 1:
            improve_days = (
                np.random.choice(ibt, 1, p=ibp)[0] * TRADING_DAYS_IN_YEAR_CONS
            )
        else:
            improve_days = ibt[0] * TRADING_DAYS_IN_YEAR_CONS

        improve_days = math.ceil(improve_days)
        finish_days_list.append(improve_days)

        # improve rate is based on the crash target and dyas needed to improve back (off line cal)
        improve_rate = (1 / (1 + target)) ** (1.0 / improve_days) - 1
        improve_rates_list.append(improve_rate)

        # configure next crash day while making sure the next crash day does not happen between the improvement of current crash
        if ci == 0:
            cday = np.random.choice(crash_years_list, 1, p=crash_year_prob_list)[0]
        else:
            while True:
                cday = np.random.choice(crash_years_list, 1, p=crash_year_prob_list)[0]
                if (
                    sum(crash_year_target_list) + cday
                ) * TRADING_DAYS_IN_YEAR_CONS > improve_back_list[ci - 1]:
                    break

        crash_year_target_list.append(cday)

        # save the date coming back to all time high in improve_back
        improve_date = (
            sum(crash_year_target_list) * TRADING_DAYS_IN_YEAR_CONS + improve_days
        )
        improve_back_list.append(improve_date)

    return {
        "finish_days_list": finish_days_list,
        "improve_rates_list": improve_rates_list,
        "crash_year_target_list": crash_year_target_list,
        "improve_back_list": improve_back_list,
        "crash_target_list": crash_target_list,
    }

In [22]:
def reformat_crash_day_target(crash_day_target_list, total_trading_days_int):
    day_total = 0
    for ci, day in enumerate(crash_day_target_list):
        if day_total < total_trading_days_int:
            day_total = day_total + day
            cutoffi = ci

    crash_day_target_list = crash_day_target_list[0:cutoffi]

    crash_day_target2_list = []
    data = 0

    for day in crash_day_target_list:
        data = data + day
        crash_day_target2_list.append(data)

    return cutoffi, crash_day_target2_list

In [23]:
# configure crash function based on dates and improve back to previous high
def crash_final_fun(
    crash_list,
    crash_day_target2_list,
    crash_target_list,
    improve_back_list,
    improve_rate_list,
):
    for ci in range(len(crash_day_target2_list)):
        cdate = int(crash_day_target2_list[ci])
        cper = crash_target_list[ci]
        improv = improve_back_list[ci]
        improvr = improve_rate_list[ci]

        for cdi in range(cdate, improv, 1):
            if cdi == cdate:
                crash_list[cdi] = crash[cdi - 1] * (1 + cper)
            else:
                crash_list[cdi] = 1 + improvr
    return crash_list

In [48]:
# increase growth rate of return to compensate the fact some days are spen in crashes
def growth_final_fun(
    growth_list,
    crash_day_target2_list,
    improve_back_list,
    growth_per_day,
):
    nr = float(total_trading_days)

    # calculate total days in crash/improve_back
    nd = 0
    for i in range(len(crash_day_target2_list)):
        nn = improve_back[i] - crash_day_target2_list[i]
        nd = nd + nn

    #
    for cc, day in enumerate(crash_day_target2_list):
        cdate = int(day)
        improv = improve_back_list[cc - 1]
        new_growth = ((1 + growth_per_day) ** nr) ** (1 / (nr - nd)) - 1
        if cc == 0:
            for cii in range(0, cdate, 1):
                growth_list[cii] = 1 + new_growth
        else:
            for i in range(improv, cdate, 1):
                growth_list[cii] = 1 + new_growth
    return growth_list

In [50]:
# Loop through growth_per_year
gain_prob_all = []

for g in range(len(growth_per_year)):
    write_log(logfile, "growth_per_year " + str(growth_per_year[g]) + " start")
    directory = "growth_per_year " + str(growth_per_year[g])
    path = os.path.join(parent_dir, directory)

    if not os.path.isdir(path):
        os.makedirs(path)

    # Loop through many simulations
    overall_gain_dca_sum = []
    overall_gain_dip_sum = []
    total = []
    gpd = growth_per_day[g]

    for sim_j in range(NUM_SIMULATIONS):
        # make a default crash funciton
        crash = []
        for i in range(total_trading_days):
            crash.append(1.0)

        # make a default growth funciton
        growth = []
        for i in range(total_trading_days):
            growth.append(1.0)

        if CRASH_ENABLE == 1:
            # get all crash related parameters
            crash_related_results = improve_time_speed_based_on_crash_value(
                improve_time,
                improve_prob,
                TRADING_DAYS_IN_YEAR,
                crash_years,
                crash_year_prob,
                NUM_YEARS,
                crash_value_prob,
                crash_values,
            )

            finish_days = crash_related_results.get("finish_days_list")
            improve_rates = crash_related_results.get("improve_rates_list")
            crash_year_target = crash_related_results.get("crash_year_target_list")
            improve_back = crash_related_results.get("improve_back_list")
            crash_target = crash_related_results.get("crash_target_list")

            crash_day_target = [x * TRADING_DAYS_IN_YEAR for x in crash_year_target]

            # reformat crash date and find the relavent first few crashes within overall date range
            cutoff_reformat = reformat_crash_day_target(
                crash_day_target, total_trading_days
            )
            cutoff = cutoff_reformat[0]
            crash_day_target2 = cutoff_reformat[1]

            # reconfigure crash results to just within range
            improve_rate = improve_rates[0:cutoff]
            improve_back = improve_back[0:cutoff]

            if improve_back[-1] > total_trading_days:
                improve_back[-1] = total_trading_days

            crash = crash_final_fun(
                crash, crash_day_target2, crash_target, improve_back, improve_rate
            )

            growth = growth_final_fun(
                growth, crash_day_target2, improve_back, gpd
            )

        crash = np.array(crash)
        growth = np.array(growth)

        # stock stdev function

        STOCK_STD_DEV_DAY = STOCK_STD_DEV / TRADING_DAYS_IN_YEAR
        pct_to_target = np.random.normal(
            1, STOCK_STD_DEV_DAY, total_trading_days
        ).round(6)

        # calculaate stock market based on stdev, crash & growth
        price = []

        for i in range(total_trading_days):
            if i == 0:
                price.append(START_MARKET_PRICE)
            else:
                price_day = price[i - 1] * pct_to_target[i] * growth[i] * crash[i]
                price.append(price_day)

        counts = range(total_trading_days)

        # put final price to data frame
        df = pd.DataFrame(
            index=range(total_trading_days),
            data={
                "total_trading_days": counts,
                "price": price,
                "pct_to_target": pct_to_target,
                "growth": growth,
                "crash": crash,
            },
        )

        # configure data table for use
        df["total_trading_days"] = df["total_trading_days"] + 1

        df["year"] = df["total_trading_days"].apply(
            lambda x: math.ceil(x / TRADING_DAYS_IN_YEAR)
        )
        df["day"] = df["total_trading_days"].apply(
            lambda x: TRADING_DAYS_IN_YEAR
            if (x % TRADING_DAYS_IN_YEAR) == 0
            else x % TRADING_DAYS_IN_YEAR
        )

        df["num_simulation"] = sim_j

        start_share = START_PROFOLIO_AMOUNT / df.iloc[0]["price"]

        df["start_share"] = start_share
        df["final_share"] = df["start_share"]
        df["cost_basis"] = START_PROFOLIO_AMOUNT

        # calculate final price for dca method
        df["add_amount"] = 0
        df["add_share"] = 0
        dca_final(df, DCA_INTERVAL_DAY, DCA_AMOUNT, overall_gain_dca_sum)

        # apr_dca = aprcalc(df, TRADING_DAYS_IN_YEAR, sim_j) # not working yet

        # dca method is the basis for dip method so copy selected columns as starting point
        selected_columns = df[
            [
                "total_trading_days",
                "price",
                "pct_to_target",
                "crash",
                "year",
                "day",
                "num_simulation",
                "start_share",
                "add_amount",
            ]
        ]

        # configure dip method columns
        dfdip = selected_columns.copy()
        dfdip["start_share"] = start_share
        dfdip["final_share"] = dfdip["start_share"]
        dfdip["cost_basis"] = START_PROFOLIO_AMOUNT
        dfdip["add_amount_temp"] = dfdip["add_amount"]
        dfdip["add_amount"] = 0

        # calculate final price for dip method
        dip_final(
            dfdip, crash_day_target2, total_trading_days, 0.9, overall_gain_dip_sum
        )

        # apr_dip = aprcalc(dfdip, TRADING_DAYS_IN_YEAR, sim_j) # not working yet

        # final table (contact all simulations together)
        df1 = df.append(dfdip, ignore_index=True)
        df1.reset_index()
        total.append(df1)
        dftotal = pd.concat(total, ignore_index=True)

        # print progress
        if NUM_SIMULATIONS <= 10:
            write_log(
                logfile,
                "finish "
                + str(sim_j + 1)
                + " simulations out of "
                + str(NUM_SIMULATIONS),
            )
        elif math.floor(100 * (sim_j + 1) / NUM_SIMULATIONS) % 10 == 0:
            per_done = str((100 * (sim_j + 1) / NUM_SIMULATIONS))
            write_log(
                logfile,
                "finish "
                + str(per_done)
                + "% simulation out of "
                + str(NUM_SIMULATIONS),
            )

    if PLOT_FIGURE == 1:
        plot_figs(dftotal, "total_trading_days", "price", path)
        plot_figs(dftotal, "total_trading_days", "final_asset", path)
        plot_figs(dftotal, "total_trading_days", "cash", path)
        plot_figs(dftotal, "total_trading_days", "add_amount", path)
        plot_figs(dftotal, "total_trading_days", "cost_basis", path)

        if CLOSE_FIGURE == 1:
            plt.close("all")

    # apr_dca.to_csv(parent_dir + "\\apr_dca.csv", index=False) # apr not working yet
    # apr_dip.to_csv(parent_dir + "\\apr_dip.csv", index=False) # apr not working yet

    plotgain = pd.DataFrame(
        {"dca": overall_gain_dca_sum, "dip": overall_gain_dip_sum,}
    )

    plotgain2 = pd.melt(
        plotgain, value_vars=["dca", "dip"], var_name="mode", value_name="gain"
    )

    if PLOT_FIGURE == 1:
        plt.figure()
        sns.histplot(
            data=plotgain2, x="gain", stat="probability", hue="mode", element="poly"
        )
        write_log(logfile, "finish gain prob plot")
        plotname = path + "\\6 gain prob.png"
        plt.savefig(plotname)

        if CLOSE_FIGURE == 1:
            plt.close("all")

    plotgain2["growth_per_year"] = growth_per_year[g]

    gain_prob_all.append(plotgain2)
    dfgain = pd.concat(gain_prob_all, ignore_index=True)
    dfgain.to_csv(parent_dir + "\\prob gain.csv", index=False)
    write_log(logfile, "growth_per_year " + str(growth_per_year[g]) + " done")

In [1]:
plot_figs(dfgain, "growth_per_year", "gain", parent_dir)

if CLOSE_FIGURE == 1:
    plt.close("all")

write_log(logfile, "completed")

NameError: name 'growth_per_year' is not defined